In [1]:
import pandas as pd

import random
import json

import os

In [2]:
# train index
# row -> [(item_id, index_time_ms, click, like, follow, forward, pltr, pwtr, pftr), ()]
def index_encoding_item(ltr_sample):
    item_id_set = set()
    for sample in ltr_sample:
        for item in sample:
            item_id_set.add(item[0])
    print ("item number=" ,len(item_id_set))
    
    item_num2id = list(item_id_set)
    item_num2id.sort()

    item_id2num = {}
    for num in range(0, len(item_id_set)):
        item_id2num[item_num2id[num]] = num

    ltr_sample_number = []
    for sample in ltr_sample:
        ltr_sample_single_item = []
        for (item_id, index_time_ms, click, like, follow, forward, pltr, pwtr, pftr) in sample:
            ltr_sample_single_item.append([item_id2num[item_id], index_time_ms, click, like, follow, forward, pltr, pwtr, pftr])
        ltr_sample_number.append(ltr_sample_single_item)

    ltr_sample = ltr_sample_number
    return ltr_sample, item_id2num

In [3]:
# filter test 
def filter_test_data(ltr_sample, item_id2num):
    new_data = []
    for sample in ltr_sample:
        new_data_single_item = []
        for (item_id, index_time_ms, click, like, follow, forward, pltr, pwtr, pftr) in sample:
            if item_id in item_id2num:
                new_data_single_item.append([item_id2num[item_id], index_time_ms, click, like, follow, forward, pltr, pwtr, pftr])
        new_data.append(new_data_single_item)

    return new_data

# write json
def write_data(path, data):
    f = open(path, 'w')
    jsObj = json.dumps(data)
    f.write(jsObj)
    f.close()

In [4]:
DIR = '../CLM/clm_pretraining_with_tenrec/dataset/Tenrec/'
origin_ltr_data_path = DIR + 'tenrec_ltr_data.json'  # NOTE: data with user time seq

path_ltr_train = DIR + 'tenrec_ltr_data_train.json'
path_ltr_test = DIR + 'tenrec_ltr_data_test.json'

# 1 load data
with open(origin_ltr_data_path) as f:
    line = f.readline()
    data = json.loads(line)
f.close()
row_num = len(data)
print ("sample number=", row_num, ", data_path=", origin_ltr_data_path)
print("data[:2]=", data[:2])

sample number= 6338947 , data_path= ../CLM/clm_pretraining_with_tenrec/dataset/Tenrec/tenrec_ltr_data.json
data[:2]= [[0, 2, 0, 0, 0, 0, 0.00061925, 0.00015654, 0.00080008], [0, 679, 1, 0, 0, 0, 0.00080245, 0.0001548, 0.0007834]]


In [5]:
# 1 N user && N user_history_list

user_set = set()
for (user_id, item_id, click, like, follow, forward, pltr, pwtr, pftr) in data:
    user_set.add(user_id)
user_num = len(user_set)
print("user_num=", user_num) 

user_num= 56751


In [6]:
user_history_list = []
for i in range(user_num):
    user_history_list.append([])
index_time_ms = 0
for (user_id, item_id, click, like, follow, forward, pltr, pwtr, pftr) in data:
    user_history_list[user_id].append([item_id, index_time_ms, click, like, follow, forward, pltr, pwtr, pftr])
    index_time_ms = index_time_ms + 1
    if index_time_ms % 1000000 == 0:
        print ("step=", index_time_ms)
# user_history_list[0]

step= 1000000
step= 2000000
step= 3000000
step= 4000000
step= 5000000
step= 6000000


In [7]:
# 2 user_history_list sort by time_ms
# row -> [（item_id, index_time_ms, click, like, follow, forward, pltr, pwtr, pftr）, (), () ..]
user_history_list_by_time = []
for row in user_history_list:
    sorted_row = sorted(row, key=lambda x: x[1])
    user_history_list_by_time.append(row)

In [8]:
# 3  count distribution
dic_action_list_length = {}
for row in user_history_list_by_time:
    length = len(row)
    try: dic_action_list_length[length] += 1
    except: dic_action_list_length[length] = 1

sorted_keys = sorted(dic_action_list_length.keys())

sorted_dict = {}
for key in sorted_keys:
    sorted_dict[key] = dic_action_list_length[key]

for key, value in sorted_dict.items():
    print ("length=", key, " num=", value)
    break
print ("len(sorted_dict)=", len(sorted_dict))

length= 10  num= 5
len(sorted_dict)= 613


In [9]:
dic_table = {}
for key, value in sorted_dict.items():
    if (key <= 10):
        try:dic_table['0-10'] += value;
        except: dic_table['0-10'] = value
    elif (key <= 20):
        try:dic_table['10-20'] += value;
        except: dic_table['10-20'] = value
    elif (key <= 50):
        try:dic_table['20-50'] += value;
        except: dic_table['20-50'] = value
    elif (key <= 100):
        try:dic_table['50-200'] += value;
        except: dic_table['50-100'] = value
    elif (key <= 200):
        try:dic_table['100-200'] += value;
        except: dic_table['100-200'] = value
    elif (key <= 300):
        try:dic_table['200-300'] += value;
        except: dic_table['200-300'] = value
    elif (key <= 400):
        try:dic_table['300-400'] += value;
        except: dic_table['300-400'] = value
    elif (key <= 500):
        try:dic_table['400-500'] += value;
        except: dic_table['400-500'] = value
    elif (key <= 1000):
        try:dic_table['500-1000'] += value;
        except: dic_table['500-1000'] = value
    elif (key <= 2000):
        try:dic_table['1000-2000'] += value;
        except: dic_table['1000-2000'] = value
    elif (key <= 2500):
        try:dic_table['2000-2500'] += value;
        except: dic_table['2000-2500'] = value
    elif (key <= 6000):
        try:dic_table['2500-6000'] += value;
        except: dic_table['2500-6000'] = value

for key, value in dic_table.items():
    print("ation_list length:", key, "num:", value, ", ratio:", round(value/user_num,6))

ation_list length: 0-10 num: 5 , ratio: 8.8e-05
ation_list length: 10-20 num: 695 , ratio: 0.012246
ation_list length: 20-50 num: 10931 , ratio: 0.192613
ation_list length: 50-100 num: 334 , ratio: 0.005885
ation_list length: 100-200 num: 17599 , ratio: 0.310109
ation_list length: 200-300 num: 4817 , ratio: 0.08488
ation_list length: 300-400 num: 1334 , ratio: 0.023506
ation_list length: 400-500 num: 461 , ratio: 0.008123
ation_list length: 500-1000 num: 188 , ratio: 0.003313
ation_list length: 1000-2000 num: 2 , ratio: 3.5e-05


In [10]:
# 4 set & step=
# row -> [（item_id, index_time_ms, click, like, follow, forward, pltr, pwtr, pftr）, (), () ..]
ltr_sample = []
candidate_set_size = 100
step = 50
for row in user_history_list_by_time:
    length = len(row)

    if length <= candidate_set_size:
        ltr_sample.append(row)
    else:
        start = 0
        end = candidate_set_size
        while end <= length:
            ltr_sample.append(row[start:end])
            start = start+step
            end = start+candidate_set_size
print("len(user_history_list_by_time)=", len(user_history_list_by_time))
print("step=", step, ", candidate_set_size=", candidate_set_size ,", len(ltr_sample)=", len(ltr_sample))
        

len(user_history_list_by_time)= 56751
step= 50 , candidate_set_size= 100 , len(ltr_sample)= 84939


In [11]:
# 5 shuffle
random.shuffle(ltr_sample)

In [12]:
# 6 spilte train:test= 9: 1
sample_number = len(ltr_sample)
train_number = int(len(ltr_sample) / 10) * 9
test_number = sample_number - train_number
print ("sample_number=",sample_number, ", train_number=", train_number, ", test_number=", test_number)

train_data_ltr_sample = ltr_sample[0:train_number]
test_data_ltr_sample = ltr_sample[train_number:]

print ("len(train_data_ltr_sample)=", len(train_data_ltr_sample), ", len(test_data_ltr_sample)=", len(test_data_ltr_sample))

sample_number= 84939 , train_number= 76437 , test_number= 8502
len(train_data_ltr_sample)= 76437 , len(test_data_ltr_sample)= 8502


In [13]:
# train_data_ltr_sample[0]

In [14]:
# 7 train index
train_data_ltr_sample, item_id2num = index_encoding_item(train_data_ltr_sample)

item number= 37453


In [15]:
# 8 test item filter
test_data_ltr_sample = filter_test_data(test_data_ltr_sample, item_id2num)
print ("test sample number=", len(test_data_ltr_sample))
# test_data_ltr_sample[0]

test sample number= 8502


In [16]:
write_data(path_ltr_train, train_data_ltr_sample)
write_data(path_ltr_test, test_data_ltr_sample)
print("write success")

write success
